In [1]:
import pandas as pd
import mxnet
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split


device = torch.device("cuda:1")
print(f"Using {device}")

2022-11-30 10:55:59.158672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Using cuda:1


In [2]:
data = pd.read_csv("../data/test.csv")

In [7]:
pred_data = data[['text', 'label']]

In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(dataset.iloc[i]['text'])]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['label']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

print('get bertmodel and vocab')
bertmodel, vocab = get_pytorch_kobert_model()


get bertmodel and vocab
using cached model. /home/mglee/VSCODE/git_folder/article_detection/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/article_detection/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
max_grad_norm = 1
log_interval = 1000
warmup_ratio = 0.1
batch_size = 1
max_len = 512

print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

pred_data_B = BERTDataset(pred_data, tok, max_len, True, False)

pred_dataloader = torch.utils.data.DataLoader(
    pred_data_B, batch_size = batch_size, num_workers = 8)


data setting
using cached model. /home/mglee/VSCODE/git_folder/article_detection/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
# for i in pred_dataloader:
#     print(i[0]) # input
#     print(i[1]) # ?
#     print(i[2]) # label
#     print(i[3]) # ?
#     break

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [12]:
PATH = '../result/kobert/'

pred_model = torch.load(PATH + 'KoBERT_1130_e10_py.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# pred_model.load_state_dict(torch.load(PATH + 'Kobert_0627_py_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

# checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
# pred_model.load_state_dict(checkpoint['model'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [13]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    #print(f"acc {train_acc} /  mi {max_indices} ==  {Y}")
    return train_acc

### Data Set만 준비한다면 여기를 계속 다른 인풋으로 돌려보면서 확인 가능

In [14]:
MAX_VALS_LST = []
MAX_IND_LST = []

out_lst = []
pred_acc = 0
pred_model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(pred_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = pred_model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    pred_acc += calc_accuracy(out, label)
    max_vals, max_indices = torch.max(out, 1)
    #MAX_VALS, MAX_IND = torch.topk(out, k =3)
    #MAX_VALS_LST.append(MAX_VALS)
    #MAX_IND_LST.append(MAX_IND)
final_acc = pred_acc / (batch_id+1)
print(f"accuracy of test data is {final_acc}")

  0%|          | 0/210 [00:00<?, ?it/s]

accuracy of test data is 0.9523809523809523


In [15]:
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(int(np.argmax(item)))

In [18]:
pred_data['pred'] = pred

In [25]:
import sklearn.metrics as metrcis
print(f"f1 score on test data is {metrcis.f1_score(pred_data['label'], pred)}")

f1 score on test data is 0.9603174603174603


In [23]:
pred_data

,text,label,pred
0,[제약] 다이어트·혈관 건강 동시에 `보이차`종근당건강 `보이미락`올여름은 유난히 ...,1.0,1
1,"호흡하는 로봇, 뇌파 탐지 밴드…꿀잠 부르는 수면 기술로 승부[이코노미조선][Int...",1.0,1
2,대구 교통·학군 중심 500가구 분양동대구 비스타동원 견본주택 개관동원개발이 25일...,1.0,1
3,[Food&Dining]야쿠르트바… 스파클링 야쿠르트… ‘국민 발효유’ 또 달라졌네...,1.0,1
4,태영건설 '에코시티 데시앙 14블록’ 분양태영건설 제공태영건설은 오는 26일 전북 ...,1.0,1
...,...,...,...
205,[2019 히트예감상품]오비맥주 '카스 미니캔'250㎖ 소용량···부담없이 맥주 청...,1.0,1
206,"대통령실 “MBC 기자-비서관 설전, 매우 심각하게 보고있다”[종합]도어스테핑 공간...",0.0,0
207,대구지하철 두류·내당역 걸어서 10분 ‘더블역세권’대구 ‘e편한세상 두류역’[서울신...,1.0,1
208,"버핏, TSMC 주식 5.4조원어치 쓸어 담았다[반도체주 투자 이례적…파운드리 기업...",0.0,1


In [20]:
pred_data.to_csv("../result/pred.csv")

기사인척하는 악성 광고성 기사 검출

In [36]:
bad_keyword = ["특집", "기획", "기자", "탐방", "취재", "인터뷰", "뉴스"]

In [42]:
bad = []

for txt in pred_data[pred_data['pred'] == 1]['text']:
    for keyword in bad_keyword:
        if keyword in txt:
            bad.append(txt)
            break

In [45]:
len(bad)

124

In [48]:
set(pred_data[pred_data['pred'] == 1]['text']) - set(bad)

{'KB자산운용, 종로타워 투자하는 부동산펀드 출시총 1120억 한도…25일까지 판매KB자산운용이 서울 종로2가 \'종로타워\'에 투자하는 펀드를 내놨다.KB자산운용이 \'KB와이즈스타부동산펀드 제2호\'를 출시하고 24~25일 이틀간 KB국민은행과 KB증권을 통해 판매한다. 이 펀드는 지하6층, 지상24층 규모의 대형업무시설인 \'종로타워\'(사진)를 기초자산으로 한다.펀드는 총 1120억원을 한도로 모집하며 6년간 폐쇄형으로 운용되며, 설정 후 90일 안에 한국거래소에 상장된다. 연 평균 예상 배당수익률은 5% 수준으로, 6개월마다 투자자들에게 분배금을 지급한다. 만기시 매각가에 따라 매각손익이 발생되는 구조여서 향후 부동산 시장상황에 따라 조기 매각할 경우 운용기간이 단축될 수 있다. 신명재 KB자산운용 부동산운용본부 상무는 "기관투자자들의 전유물이었던 코어 부동산 상품을 공모화 했다" 며 "종로타워는 서울의 핵심 지역에 위치한 빌딩으로 타 부동산에 비해 안정적으로 임대료 수입을 확보할 수 있고, 환가성이 높아 매각시 매각차익도 기대된다"고 말했다.KB자산운용은 지난해 1월 이현승 대표이사가 취임한 이후 국내외 부동산펀드 시장에서 두각을 나타내고 있다. 지난해 SK증권빌딩(2950억원), KDB생명빌딩(4220억원), 서안성 물류센터(610억원), 스카이파크호텔(1960억원) 등 국내 부동산에만 1조원 가량을 투자했다.앞서 KB자산운용 지난 2월 옛 KB국민은행 명동본점 부지 개발사업에 중순위 대출을 실행하는 공모펀드 \'KB와이즈스타부동산펀드 제1호\'(모집액 750억원)를 출시, 10분 만에 완판하는 성과를 올리기도 했다.▶ 헉! 소리나는 스!토리 뉴스 [헉스]▶ 클릭해, 뉴스 들어간다 [뉴스쿨]▶ 세상에 이런일이 [fn파스]※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지nannan',
 '기보, 식품진흥원과 식품中企 기술혁신 \'맞손\'차세대 기술거래 플랫폼 연계…식품산업 혁신성장 기여기술보증기금(이사장 김종호, 이하 기보)은 한국식품산업클러스